In [44]:
import pandas as pd
import numpy as np
import plotly.express as px
import streamlit as st
from ipywidgets import interact, Dropdown, IntSlider

---

## Chargement des données

In [45]:
# --- Chargement des données nettoyées (préparées dans le Notebook 1) ---
# dvf
df = pd.read_csv("dvf_idf_clean.csv")


### Widget 2 : Carte avec score de liquidité de marché

In [46]:
# Sécuriser la date
df["date_mutation"] = pd.to_datetime(df["date_mutation"], errors="coerce")
df = df.dropna(subset=["date_mutation"])

# Colonnes minimales utiles
needed = ["date_mutation", "code_commune", "nom_commune", "latitude", "longitude"]
assert all(c in df.columns for c in needed), "Colonnes manquantes pour l'indice."

df["mois"] = df["date_mutation"].dt.to_period("M").astype(str)

# Ventes par commune × mois
mth = (df.groupby(["code_commune","nom_commune","mois"], as_index=False)
               .size()
               .rename(columns={"size":"nb_ventes_mois"}))

# On veut 12 mois même si 0 vente 
all_months = pd.period_range("2024-01", "2024-12", freq="M").astype(str)
pivot = (mth.pivot_table(index=["code_commune","nom_commune"],
                         columns="mois", values="nb_ventes_mois", fill_value=0)
            .reindex(columns=all_months, fill_value=0))

pivot.head()

feat = pivot.copy()
feat["nb_ventes_2024"] = feat.sum(axis=1)
feat["nb_mois_actifs"] = (feat[all_months] > 0).sum(axis=1)

# CV mensuel (std/mean) — si mean=0 → CV=0 pour éviter NaN
mean = feat[all_months].mean(axis=1)
std  = feat[all_months].std(axis=1)
feat["cv_mensuel"] = np.where(mean > 0, (std / (mean + 1e-9)), 0.0)

# Stabilité = 1 - CV normalisé
# Normalisation min-max sur l'ensemble des communes
cv_min, cv_max = feat["cv_mensuel"].min(), feat["cv_mensuel"].max()
feat["cv_norm"] = (feat["cv_mensuel"] - cv_min) / (cv_max - cv_min + 1e-9)
feat["stabilite"] = 1 - feat["cv_norm"]  # plus stable → score ↑

# Remettre en DataFrame plat
feat = feat.reset_index()
feat = feat[["code_commune","nom_commune","nb_ventes_2024","nb_mois_actifs","stabilite"]]

# Rangs percentile (0..1)
feat["rk_volume"] = feat["nb_ventes_2024"].rank(pct=True)
feat["rk_mois"]   = feat["nb_mois_actifs"].rank(pct=True)
feat["rk_stab"]   = feat["stabilite"].rank(pct=True)

# Pondérations 
w_vol, w_mois, w_stab = 0.60, 0.20, 0.20

feat["score_liquidite_2024"] = (w_vol*feat["rk_volume"] +
                                w_mois*feat["rk_mois"] +
                                w_stab*feat["rk_stab"]) * 100

# Classe qualitative
feat["classe_liquidite"] = pd.cut(
    feat["score_liquidite_2024"],
    bins=[-1,33,66,101],
    labels=["Illiquide","Intermédiaire","Liquide"],
    right=False
)


In [ ]:
feat.sort_values("score_liquidite_2024", ascending=False).head(10)


,code_commune,nom_commune,nb_ventes_2024,nb_mois_actifs,stabilite,rk_volume,rk_mois,rk_stab,score_liquidite_2024,classe_liquidite
17,75118,Paris 18e Arrondissement,2624.0,12,0.972278,0.999202,0.82482,0.988029,96.209098,Liquide
14,75115,Paris 15e Arrondissement,2817.0,12,0.971362,1.000000,0.82482,0.984038,96.177175,Liquide
15,75116,Paris 16e Arrondissement,2138.0,12,0.971664,0.996808,0.82482,0.985634,96.017558,Liquide
16,75117,Paris 17e Arrondissement,2314.0,12,0.965861,0.998404,0.82482,0.967279,95.746209,Liquide
10,75111,Paris 11e Arrondissement,2224.0,12,0.966088,0.997606,0.82482,0.968875,95.730247,Liquide
1011,93048,Montreuil,948.0,12,0.975053,0.988029,0.82482,0.990423,95.586592,Liquide
957,92004,Asnières-sur-Seine,1068.0,12,0.967643,0.990423,0.82482,0.976057,95.442937,Liquide
1084,95018,Argenteuil,853.0,12,0.972985,0.986034,0.82482,0.988827,95.434956,Liquide
18,75119,Paris 19e Arrondissement,1407.0,12,0.965049,0.992817,0.82482,0.966480,95.395052,Liquide
9,75110,Paris 10e Arrondissement,1334.0,12,0.966279,0.991221,0.82482,0.970471,95.379090,Liquide


In [34]:
centroids = (df.groupby(["code_commune","nom_commune"], as_index=False)
                    .agg(latitude=("latitude","median"),
                         longitude=("longitude","median")))

liq_2024 = feat.merge(centroids, on=["code_commune","nom_commune"], how="left")
liq_2024.shape


(1253, 12)

In [ ]:
df = liq_2024.copy()   # dataset liquidité construit avant

@interact(
    score_min = IntSlider(min=0, max=100, step=5, value=40),
    ventes_min = IntSlider(min=0, max=df["nb_ventes_2024"].max(), step=10, value=200)
)
def plot_liquidite(score_min, ventes_min):
    
    tmp = df[(df["score_liquidite_2024"] >= score_min) &
             (df["nb_ventes_2024"] >= ventes_min)]
    
    fig = px.scatter_mapbox(
        tmp,
        lat="latitude",
        lon="longitude",
        size="nb_ventes_2024",
        color="score_liquidite_2024",
        hover_name="nom_commune",
        hover_data=["nb_ventes_2024","nb_mois_actifs","stabilite"],
        zoom=8,
        height=600,
        center={"lat":48.85,"lon":2.35},
        color_continuous_scale="Viridis"
    )
    
    fig.update_layout(mapbox_style="open-street-map")
    fig.show()


interactive(children=(IntSlider(value=40, description='score_min', step=5), IntSlider(value=200, description='…

In [51]:


# df doit contenir :
# latitude, longitude, score_liquidite_2024, nb_ventes_2024, nom_commune, stabilite

df_map = liq_2024.copy()

# Palette personnalisée
color_scale = [
    [0.0, "#d7191c"],   # rouge
    [0.33, "#fdae61"],  # orange
    [0.66, "#abdda4"],  # vert clair
    [1.0, "#2b83ba"],   # bleu
]

fig = px.scatter_mapbox(
    df_map,
    lat="latitude",
    lon="longitude",
    color="score_liquidite_2024",
    size="nb_ventes_2024",
    size_max=25,
    color_continuous_scale=color_scale,
    hover_name="nom_commune",
    hover_data={
        "score_liquidite_2024": True,
        "nb_ventes_2024": True,
        "stabilite": True,
        "latitude": False,
        "longitude": False,
    },
    zoom=9,
    height=650,
)

# Style de carte
fig.update_layout(
    mapbox_style="carto-positron",
    title="Indice de liquidité — Carte interactive (2024)",
    title_x=0.5,
    coloraxis_colorbar=dict(
        title="Score",
        ticks="outside",
        tickvals=[0, 25, 50, 75, 100],
        lenmode="fraction",
        len=0.4
    ),
    margin=dict(l=0, r=0, t=60, b=0),
)

# Cercle noir fin autour des points
fig.update_traces(
    marker=dict(
        sizemode="area",
        sizeref=2.*max(df_map["nb_ventes_2024"])/(25**2),
    )
)


fig.show()


C:\Users\youda\AppData\Local\Temp\ipykernel_18708\3410080622.py:14: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [ ]:
from ipywidgets import interact, IntSlider
import plotly.express as px

df_map = liq_2024.copy()

@interact(
    score_min = IntSlider(min=0, max=100, step=5, value=60, description="Score min"),
    ventes_min = IntSlider(min=0, max=df_map["nb_ventes_2024"].max(), step=10, value=100, description="Ventes min")
)
def plot_liquidite(score_min, ventes_min):

    tmp = df_map[
        (df_map["score_liquidite_2024"] >= score_min) &
        (df_map["nb_ventes_2024"] >= ventes_min)
    ].copy()

    # Ajustement taille marqueur
    tmp["pt_size"] = (tmp["nb_ventes_2024"] / tmp["nb_ventes_2024"].max()) * 25 + 5

    fig = px.scatter_mapbox(
        tmp,
        lat="latitude",
        lon="longitude",
        color="score_liquidite_2024",
        size="pt_size",
        hover_name="nom_commune",
        hover_data=["nb_ventes_2024", "nb_mois_actifs", "stabilite"],
        zoom=9,
        height=650,
        center={"lat": 48.85, "lon": 2.35},
        color_continuous_scale="Turbo",   # ✅ meilleure lisibilité
    )

    fig.update_layout(
        mapbox_style="carto-positron",
        title="Indice de liquidité — Carte interactive (2024)",
        title_x=0.5,
        margin=dict(l=0, r=0, t=60, b=0),
        coloraxis_colorbar=dict(
            title="Liquidité",
            thickness=15,
            len=0.75,
        )
    )

    fig.update_traces(
        marker=dict(
            opacity=0.85,
        )
    )

    fig.show()


interactive(children=(IntSlider(value=60, description='Score min', step=5), IntSlider(value=100, description='…

## Exploration interactive par budget

### Widget 2.1 : Explorateur de budget (interactif - 1 à 3 paramètres)
<!-- COMPLÉTEZ ICI: Décrivez les paramètres interactifs -->
<!-- Exemple: -->
<!-- - Slider pour le budget (min-max) -->
<!-- - Menu déroulant pour le type de bien -->
<!-- - Slider pour la surface minimale -->

### Widget 2.2 : Calculateur de rentabilité locative (interactif - 1 à 3 paramètres)
<!-- COMPLÉTEZ ICI: Décrivez les paramètres -->
<!-- Exemple: Prix d'achat, loyer mensuel estimé, charges, etc. -->


In [ ]:
# CODEZ ICI: Créer les inputs interactifs

# CODEZ ICI: Fonction de calcul du ROI, rendement brut, rendement net

# CODEZ ICI: Affichage des résultats avec visualisation

---

## Analyse géographique

### Widget 3.1 : Comparaison régionale (interactif - 1 à 3 paramètres)

<!-- COMPLÉTEZ ICI: Décrivez la comparaison attendue -->
<!-- Exemple: Comparer 2-3 régions/villes sur plusieurs critères -->


In [ ]:
# CODEZ ICI: Sélecteur de régions/villes à comparer

# CODEZ ICI: Calcul des métriques de comparaison

# CODEZ ICI: Visualisation comparative (barres, radar chart, etc.)

### Widget 3.2 : Carte interactive des prix (complexe - carte + paramètres)

<!-- COMPLÉTEZ ICI: Décrivez les fonctionnalités de la carte -->
<!-- Exemple: -->
<!-- - Carte choroplèthe des prix moyens -->
<!-- - Filtres par type de bien -->
<!-- - Pop-up avec détails au clic -->
<!-- - Layers pour différentes métriques -->

In [ ]:
# CODEZ ICI: Créer la carte interactive avec Folium ou Plotly

# CODEZ ICI: Ajouter les paramètres de filtrage

# CODEZ ICI: Afficher la carte


---

## Système de recommandation

### Widget 4.1 : Détecteur d'opportunités (complexe - plus de 3 paramètres)

<!-- COMPLÉTEZ ICI: Décrivez tous les paramètres du système de recommandation -->
<!-- Exemple: -->
<!-- - Budget min/max -->
<!-- - Type(s) de bien souhaité(s) -->
<!-- - Rendement minimum attendu -->
<!-- - Région(s) préférée(s) -->
<!-- - Niveau de risque acceptable -->
<!-- - etc. -->

In [ ]:
# CODEZ ICI: Créer tous les widgets de paramétrage

# CODEZ ICI: Fonction de scoring/recommandation

# CODEZ ICI: Affichage des Top N opportunités recommandées

# CODEZ ICI: Visualisation des recommandations (tableau + carte/graphique)

---

## Tableaux de bord personnalisés

### Widget 6.1 : Dashboard personnalisé

<!-- COMPLÉTEZ ICI: Décrivez les éléments du dashboard -->
<!-- Un dashboard qui combine plusieurs métriques importantes pour le persona -->


In [ ]:
# CODEZ ICI: Créer un dashboard multi-graphiques

# CODEZ ICI: Possibilité de personnaliser les métriques affichées

---

## Conclusions et recommandations

### Synthèse des analyses
<!-- COMPLÉTEZ ICI: Rédigez les conclusions principales -->
<!-- Basées sur les analyses effectuées dans les sections précédentes -->

### Recommandations pour l'investisseur
<!-- COMPLÉTEZ ICI: Listez les recommandations concrètes -->
<!-- Exemple: -->
<!-- 1. Les 3 meilleures zones identifiées -->
<!-- 2. Le type de bien optimal -->
<!-- 3. Les zones à éviter -->
<!-- 4. Conseils stratégiques -->

### Points d'attention
<!-- COMPLÉTEZ ICI: Listez les limites et précautions -->
<!-- Exemple: Limites des données, facteurs non pris en compte, etc. -->

---

## Annexes

### Méthodologie
<!-- COMPLÉTEZ ICI: Expliquez votre méthodologie -->
<!-- - Comment les données ont été nettoyées -->
<!-- - Formules de calcul utilisées -->
<!-- - Choix techniques justifiés -->

### Sources et références
<!-- COMPLÉTEZ ICI: Listez toutes vos sources -->
<!-- - Liens vers les datasets -->
<!-- - Documentation des APIs utilisées -->
<!-- - Références bibliographiques si applicable -->

---

## Guide d'utilisation

### Comment utiliser ce notebook
<!-- COMPLÉTEZ ICI: Instructions pour l'utilisateur final -->
<!-- 1. Exécuter les cellules dans l'ordre -->
<!-- 2. Comment interagir avec les widgets -->
<!-- 3. Comment interpréter les résultats -->
<!-- etc. -->

### Dépendances requises

In [ ]:
# Liste des bibliothèques nécessaires
# COMPLÉTEZ ICI: Listez toutes les bibliothèques et leurs versions
# pandas==X.X.X
# matplotlib==X.X.X
# etc.

---

**Projet développé par :**
- Ashley OHNONA
- Harisoa RANDRIANASOLO
- Fairouz YOUDARENE
- Jennifer ZAHORA

**Date :** <!-- COMPLÉTEZ ICI: Date -->

**Version :** 1.0